In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn. ensemble import EasyEnsembleClassifier
import warnings
warnings.filterwarnings('ignore')

In [2]:
# load in CSV files
LoanStats = pd.read_csv("LoanStats1.csv")
Lending_Data = pd.read_csv("lending_data.csv")
LoanStats.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,dti,loan_status,delinq_2yrs,inq_last_6mths,open_acc,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,10500,0.1719,375.35,RENT,66000.0,27.24,low_risk,0,0,8,...,0,3,85.7,100.0,0,0,65687,38199,2000,61987
1,25000,0.2000,929.09,MORTGAGE,105000.0,20.23,low_risk,0,0,17,...,0,6,91.2,50.0,1,0,271427,60641,41200,49197
2,20000,0.2000,529.88,MORTGAGE,56000.0,24.26,low_risk,0,0,8,...,0,4,66.7,50.0,0,0,60644,45684,7500,43144
3,10000,0.1640,353.55,RENT,92000.0,31.44,low_risk,0,1,10,...,0,1,100.0,50.0,1,0,99506,68784,19700,76506
4,22000,0.1474,520.39,MORTGAGE,52000.0,18.76,low_risk,0,1,14,...,0,1,100.0,0.0,0,0,219750,25919,27600,20000


In [3]:
# Creating a Fit label for homeowner column
from sklearn.preprocessing import LabelEncoder
Label_Encoder=LabelEncoder()
Label_Encoder.fit(LoanStats["home_ownership"])
LoanStats["home_ownership"] = Label_Encoder.transform(LoanStats["home_ownership"])
LoanStats.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,dti,loan_status,delinq_2yrs,inq_last_6mths,open_acc,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,10500,0.1719,375.35,3,66000.0,27.24,low_risk,0,0,8,...,0,3,85.7,100.0,0,0,65687,38199,2000,61987
1,25000,0.2000,929.09,1,105000.0,20.23,low_risk,0,0,17,...,0,6,91.2,50.0,1,0,271427,60641,41200,49197
2,20000,0.2000,529.88,1,56000.0,24.26,low_risk,0,0,8,...,0,4,66.7,50.0,0,0,60644,45684,7500,43144
3,10000,0.1640,353.55,3,92000.0,31.44,low_risk,0,1,10,...,0,1,100.0,50.0,1,0,99506,68784,19700,76506
4,22000,0.1474,520.39,1,52000.0,18.76,low_risk,0,1,14,...,0,1,100.0,0.0,0,0,219750,25919,27600,20000


In [4]:
# Create our Features
X = LoanStats.drop(columns = "loan_status")

Y = LoanStats["loan_status"]
X

,loan_amnt,int_rate,installment,home_ownership,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,10500,0.1719,375.35,3,66000.0,27.24,0,0,8,0,...,0,3,85.7,100.0,0,0,65687,38199,2000,61987
1,25000,0.2000,929.09,1,105000.0,20.23,0,0,17,1,...,0,6,91.2,50.0,1,0,271427,60641,41200,49197
2,20000,0.2000,529.88,1,56000.0,24.26,0,0,8,0,...,0,4,66.7,50.0,0,0,60644,45684,7500,43144
3,10000,0.1640,353.55,3,92000.0,31.44,0,1,10,1,...,0,1,100.0,50.0,1,0,99506,68784,19700,76506
4,22000,0.1474,520.39,1,52000.0,18.76,0,1,14,0,...,0,1,100.0,0.0,0,0,219750,25919,27600,20000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,10000,0.1502,346.76,3,26000.0,9.60,0,0,9,0,...,0,3,80.0,0.0,0,0,20625,6798,11300,5425
68813,12000,0.2727,368.37,3,63000.0,29.07,0,0,8,0,...,0,5,96.2,0.0,0,0,87939,60350,13500,62939
68814,5000,0.1992,185.62,1,52000.0,14.86,0,0,5,1,...,0,3,100.0,0.0,1,0,30592,18611,3600,18492
68815,40000,0.0646,1225.24,1,520000.0,9.96,0,1,21,0,...,0,3,98.2,12.5,0,0,1033574,95958,100800,78634


In [5]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,...,0.052138,2.219423,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,...,0.390633,1.897432,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,...,0.000000,0.000000,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,...,0.000000,1.000000,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,...,0.000000,2.000000,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,...,0.000000,3.000000,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,...,18.000000,19.000000,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06


In [6]:
# Check the balance of our target values
Y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [7]:
# Split the X and y into X_train, X_test, y_train, y_test
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X,
                                                   Y,
                                                   random_state=1,
                                                   stratify=Y)
X_Train.shape

(51612, 77)

In [8]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
Scalers = StandardScaler()

In [9]:
# Fitting the scaler
X_Scaler = Scalers.fit(X_Train)

In [10]:
# Scaler Data
X_Train_Scaled =X_Scaler.transform(X_Train)
X_Test_Scaled = X_Scaler.transform(X_Test)

In [11]:
# Balanced Random Forest Classifier

# Resample the training data with the BalancedRandomForestClassifier
Balanced_Classifier = BalancedRandomForestClassifier(random_state=1)
Balanced_Classifier.fit(X_Train_Scaled, Y_Train)

BalancedRandomForestClassifier(random_state=1)

In [12]:
# Calculated the balanced accuracy score
Y_Balanced_Classifier_Pred = Balanced_Classifier.predict(X_Test_Scaled)
Balanced_Accuracy_Score_Balanced_Classifier = balanced_accuracy_score(Y_Test, Y_Balanced_Classifier_Pred)
Balanced_Accuracy_Score_Balanced_Classifier

0.6891918569281781

In [13]:
# Display the confusion matrix

Confusion_Matrix_Balanaced_Classifier = confusion_matrix(Y_Test, Y_Balanced_Classifier_Pred)
Confusion_Matrix_Balanaced_Classifier_df = pd.DataFrame(Confusion_Matrix_Balanaced_Classifier, index=["Actual High", "Actual Low"], columns = ["Predicted High", "Predicted Low"])
Confusion_Matrix_Balanaced_Classifier_df

,Predicted High,Predicted Low
Actual High,49,38
Actual Low,3164,13954


In [14]:
# Print the imbalanced classification report

classification_report_imbalanced(Y_Test, Y_Balanced_Classifier_Pred)

'                   pre       rec       spe        f1       geo       iba       sup\n\n  high_risk       0.02      0.56      0.82      0.03      0.68      0.45        87\n   low_risk       1.00      0.82      0.56      0.90      0.68      0.47     17118\n\navg / total       0.99      0.81      0.56      0.89      0.68      0.47     17205\n'

In [15]:
# List the features sorted in descending order by feature importance
Feature_Importance = Balanced_Classifier.feature_importances_
Feature_Importance

array([0.01242353, 0.03353158, 0.01765859, 0.00249933, 0.0171785 ,
       0.0179281 , 0.00464357, 0.00645498, 0.01190593, 0.00176944,
       0.01601428, 0.0119631 , 0.01660481, 0.0158435 , 0.04169818,
       0.05469087, 0.08493512, 0.07058874, 0.00727041, 0.        ,
       0.        , 0.05084559, 0.00023692, 0.        , 0.        ,
       0.00269469, 0.01526142, 0.00482528, 0.00797238, 0.00434151,
       0.00865535, 0.01339823, 0.01510571, 0.01810675, 0.00607087,
       0.00608025, 0.0179958 , 0.01455153, 0.01463568, 0.00795508,
       0.0083723 , 0.01283187, 0.01103653, 0.01552815, 0.01351871,
       0.01739769, 0.00024119, 0.        , 0.013418  , 0.015385  ,
       0.01004699, 0.0113786 , 0.00677085, 0.0111666 , 0.01936311,
       0.00436746, 0.01030892, 0.00747354, 0.00967041, 0.00941458,
       0.01071971, 0.00931284, 0.01045812, 0.00943809, 0.01047145,
       0.        , 0.        , 0.00140574, 0.00649372, 0.00905883,
       0.00763639, 0.00215383, 0.        , 0.01584519, 0.01315

In [16]:
# Easy Ensemble Classifier
Easy_Ensemble_Classifier = EasyEnsembleClassifier(random_state=1)
Easy_Ensemble_Classifier.fit(X_Train_Scaled, Y_Train)

EasyEnsembleClassifier(random_state=1)

In [17]:
# Calculated the balanced accuracy score
Y_Easy_Ensemble_Classifier_Pred = Easy_Ensemble_Classifier.predict(X_Test_Scaled)
Balanced_Accuracy_Easy_Ensemble_Classifier=balanced_accuracy_score(Y_Test, Y_Balanced_Classifier_Pred)
Balanced_Accuracy_Easy_Ensemble_Classifier

0.6891918569281781

In [20]:
# Display the confusion matrix
Confusion_Matrix_Easy_Ensemble_Classifier=confusion_matrix(Y_Test, Y_Easy_Ensemble_Classifier_Pred)
Confusion_Matrix_Easy_Ensemble_Classifier_df = pd.DataFrame(Confusion_Matrix_Easy_Ensemble_Classifier, index=["Actual High", "Actual Low"], columns=['Predicted High', "Predicted Low"])
Confusion_Matrix_Easy_Ensemble_Classifier_df

,Predicted High,Predicted Low
Actual High,61,26
Actual Low,3459,13659


In [21]:
# Print the imbalanced classification report
classification_report_imbalanced(Y_Test, Y_Easy_Ensemble_Classifier_Pred)

'                   pre       rec       spe        f1       geo       iba       sup\n\n  high_risk       0.02      0.70      0.80      0.03      0.75      0.55        87\n   low_risk       1.00      0.80      0.70      0.89      0.75      0.56     17118\n\navg / total       0.99      0.80      0.70      0.88      0.75      0.56     17205\n'

In [ ]:
print(f"From the data calculated above all models had the same balanced accuracy score:")
print(f" The Balanced Random Forest Classifier had a total recal score of 0.81, compared to Easy Ensemble Classifier of .8")
print(f" The Balanced Random Forest Classifier had the best geometric mean score) 
